In [93]:
import pandas as pd
import numpy as np

In [140]:
'''
This function adds a column at each row with the user_id that "infected" the row and NaN if it wasn't infected by anyone (seed)
'''

def get_father(log):
    for index, row in log.iterrows():
        id_post_origin = row['id_tweet_origin'] #change id_tweet_origin to id_post_origin if needed
        if id_post_origin != 0:
            infected_by = int(log[log['id_tweet'] == id_post_origin]['id_user']) #change id_tweet to id_post if needed
            log.loc[index,'infected_by'] = infected_by
    return

In [134]:
'''
Input: Paths of the two database files
Output: Dataframe sorted by time and with a column "infected_by"
'''
def trating_data(instagram_path, twitter_path):
    #importing instagram data
    dfs_i = pd.read_excel(instagram_path, sheet_name= None, date_parser  = int)
    i_post, i_user = dfs_i[list(dfs_i.keys())[1]] , dfs_i[list(dfs_i.keys())[2]]
    #i_user.rename(columns={'nb_posts':'nb_tweets'}, inplace=True)
    #i_post.rename(columns={'id_post_origin':'id_tweet_origin'}, inplace=True)
    get_father(i_post)
    
    #importing twitter data
    dfs_t = pd.read_excel(twitter_path, sheet_name= None, date_parser  = int)
    t_post, t_user = dfs_t[list(dfs_t.keys())[1]] , dfs_t[list(dfs_t.keys())[2]]
    t_user.rename(columns={'id_tweet':'id_post'}, inplace=True)
    get_father(t_post)

    columns = ['id_user','id_post', 'infected_by', 'date','time','half_day']
    
    df = pd.concat([i_post[columns], t_post[columns]], axis=0)
    
    return df

In [135]:
instagram_path = "C:/Users/menoci/Desktop/modelisation/Propagation_Virale/INTEGRATION_WEEK/instagram_dataset.xlsx"
twitter_path = "C:/Users/menoci/Desktop/modelisation/Propagation_Virale/INTEGRATION_WEEK/twitter_dataset.xlsx"

## ERXCTRYVGUBHINJO?K

In [141]:
#importing instagram data
dfs_i = pd.read_excel("C:/Users/menoci/Desktop/modelisation/Propagation_Virale/INTEGRATION_WEEK/instagram_dataset.xlsx", sheet_name= None, date_parser  = int)
i_post, i_user = dfs_i[list(dfs_i.keys())[1]] , dfs_i[list(dfs_i.keys())[2]]
i_post.rename(columns={'id_post':'id_tweet'}, inplace=True)
i_post.rename(columns={'id_post_origin':'id_tweet_origin'}, inplace=True)
#get_father(i_post)

In [142]:
'''
Input: Log (t_post) and the seeds (seed_set)
Output: the DNI list
Measures how good are the seeds (seed_set) chosen given the trace (log) = t_post

'''
def get_dni(log, seed_set):
    t_post = log.copy()
    t_post['time'] = t_post["time"].astype('|S') #already done
    t_post = t_post.sort_values(by=['date', 'half_day','time']) #already done
    get_father(t_post) #already done
    t_post.reset_index(drop = True, inplace = True) #the smaller the index, sooner the post was shared
    t_post['index'] = t_post.index
    
    DNI = set({})
    for node in seed_set:
        index_node = t_post[ t_post['id_user']==node].index.values[0] #didn't find an easier way to get the index
        to_visit = [node]
        while to_visit:
            x = to_visit.pop()
            if x not in DNI:
                for infected_by_x in list(t_post[t_post['infected_by']== x ][t_post['index'] >= index_node]['id_user']):
                    if infected_by_x not in DNI:
                        to_visit.append(infected_by_x)
                DNI.add(x)
    return DNI
    

In [143]:
"Let's check if the code is working well by trying the seed set as being the real one with twitter's log"

# importing twitter's data:
dfs_t = pd.read_excel("C:/Users/menoci/Desktop/modelisation/Propagation_Virale/INTEGRATION_WEEK/twitter_dataset.xlsx", sheet_name= None, date_parser  = int)
t_post, t_user = dfs_t[list(dfs_t.keys())[1]] , dfs_t[list(dfs_t.keys())[2]]

seed_set = [3003097,6013435,6027974,1953787,9834565,3027418]
DNI = get_dni(t_post, seed_set)

C:\Users\menoci\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [144]:
len(DNI)

4098

In [86]:
"let's try with a smaller set and see the result"
seed_set = [3003097,6013435]
DNI2 = get_dni(t_post, seed_set)

C:\Users\menoci\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [87]:
len(DNI2)    

1116

In [114]:
"Let's try Instagram's data"
seed_set = [672702,474227,587566,483543]
DNI_insta = get_dni(i_post, seed_set)
len(DNI_insta)

C:\Users\menoci\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:21: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


3046

In [116]:
len(set(i_post.id_user))

3046

In [96]:
t_post

,id_user,id_tweet,date,time,half_day,location,views,likes,comments,retweets,id_tweet_origin,link_clicks,donation_tag,donation_val,house_buy
0,9834565,18271010,2017-11-09,09:41:00,am,Montreuil,62,24,2,16,0,False,False,0,False
1,1953787,55199327,2017-11-09,08:17:00,am,Reims,148,66,8,19,0,False,False,0,False
2,6013435,81242015,2017-11-09,05:18:00,am,Paris,78,23,0,10,0,False,False,0,False
3,3027418,68398591,2017-11-09,10:03:00,am,Villeurbanne,208,62,8,20,0,False,False,0,False
4,6027974,94580818,2017-11-09,08:10:00,am,Paris,23,9,0,18,0,False,False,0,False
5,3003097,48168379,2017-11-09,02:53:00,am,Marseille,26,9,1,20,0,False,False,0,False
6,9266081,50443972,2017-11-09,02:55:00,pm,Angers,93,23,2,9,48168379,False,False,0,False
7,5265603,53869774,2017-11-09,04:26:00,pm,Strasbourg,78,7,7,6,18271010,False,False,0,False
8,8820586,94306511,2017-11-09,07:23:00,pm,Tourcoing,35,15,0,3,81242015,False,False,0,False
9,3776734,95227566,2017-11-09,02:12:00,pm,Saint-Denis,83,33,0,8,18271010,False,False,0,False


In [103]:
"""
Let's try with the full data, twitter + instagram
"""


C:\Users\menoci\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:20: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [ ]:
len(DNI)